In [1]:
import itertools
import numpy as np
from scipy.special import comb


class ReedMuller:
    def __init__(self, m, r):
        self.m = m
        self.r = r

    def encode(self, iwd):
        cwd = np.zeros((2**self.m,))
        for i, t in enumerate(itertools.product([0, 1], repeat=self.m)):
            cwd[i] = self._evaluate_polynomial(iwd, t)
        return cwd.astype(np.int)

    def cwd_length(self):
        # Calculate codeword length
        return 2 ** self.m

    def iwd_length(self):
        # Calculate information word length
        iwd_length = 0
        for i in range(self.r + 1):
            iwd_length += comb(self.m, i)
        return int(iwd_length)

    #based on https://courses.cs.washington.edu/courses/cse533/06au/lecnotes/lecture8.pdf
    def decode(self, received_codeword):
        m = self.m
        r  = self.r
        iwd_hat = np.zeros(self.iwd_length())
        while r >= 0:
            basis = np.array([2**i for i in range(m - r)])
            
            for idx in itertools.combinations(range(m), m - r):
                b = np.zeros(2**(m - r))
                for i, j in enumerate(itertools.product(range(2), repeat=m)): 
                    if r is not m: 
                        indx = np.array(j)[np.array(idx)]@basis
                        b[indx] += received_codeword[i] 
                        b[indx] -= self._evaluate_polynomial(iwd_hat,j)
                    else:
                        b[0] += self._evaluate_polynomial(received_codeword,j)
                if ((b%2).sum() > (b%2).shape[0]/2) == True:
                    cs = 1
                else:
                    cs = 0
                iwd_hat[self.to_index(idx)] += cs
            r -= 1
        return iwd_hat

    def _evaluate_polynomial(self, poly_coeff, x):
        poly_coef_id = 0  # index for polynomial coefficient
        poly_val = 0
        # Iterate through the monomial power
        for monomial_power in range(self.r + 1):
            # Iterate all fixed power monomials
            for t in itertools.combinations(range(self.m), monomial_power):
                # Evaluate monomial value
                mono_val = poly_coeff[poly_coef_id]
                for j in t:
                    mono_val *= x[j]
                poly_val += mono_val
                poly_coef_id += 1
        return np.mod(poly_val, 2)


    def to_index(self, indexes):
        all_indexes = np.arange(self.m)
        com_indexes = []
        index = 0
        for i in all_indexes:
            if i not in indexes:
                com_indexes.append(i)
        for i in range(len(com_indexes)):
            index += comb(self.m, i)
        for i,t in enumerate(itertools.combinations(range(self.m), len(com_indexes))):
            if list(t) == com_indexes:
                index += i
                break
        return int(index) 
    

def array_to_string(array):
    string = ''
    for item in array:
        string += str(int(item))
    return string


def msg_to_cdw(msg):
    c = ''
    for item in msg:
        c += (format(int(item,16),'04b'))[::-1] 
    return c

def bin_to_hex(vector):
    final = ''
    array = [(array_to_string(vector))[i:i+4] for i in range(0,len(vector),4)]
    for item in array:
        final += hex(int((item)[::-1],2))[2:]
    return final

In [2]:
message = '162ad9b04a86e37a649b97c20b0461c49d6df8a2f20df1a4e0d34620bc8083eae04ff8579b5dd91fef7c5dcea8adbfba02a2d6894a7cc40da73b2645d326f7fd6cdd8893aa14283cd8c3aae4ee0505bb7ef330e8ed6fc512a37b7b093f177efca4680da74c4c8c15cb080b51d323851345dc20df085dfb37e97fe5eaab0ece0d20ced9aec21f6e2a10cd16ad9b75c84015a1d0fd522ac223d5adeff1fb4f94462f7a62382983570da2c4e0890e977f165208ba1068cdb319b6205137264ff2948eb7284b69503053b44e48e835303669f39099a0d7b4427b0a55c5c0b717877de9eaa8014329c101801afd32bcb0025b2919a4c126809764d98f976bbf80cd58ff8b12cc2d6a0c1e77033f1e0f487499b80f5ab8a62277599ad222c02e555a74a1b6b057dcf8c21640ceaed03d80236e26643785cdbc2cad6d497ca88691987f'
c = msg_to_cdw(message)
constr = ReedMuller(8,3)
cdw = np.array([int(c[i]) for i in range(len(c))])
split = np.hsplit(cdw,10)

vector = {'inf': np.array([]),
         'cdw': np.array([]),
         'err': np.array([])
        }
for item in split :
    current = constr.decode(item)
    vector['inf'] = np.hstack([vector['inf'],current])
    vector['cdw'] = np.hstack([vector['cdw'], constr.encode(current)])
vector['err'] = (vector['cdw'] - cdw)%2

In [3]:
print('For information vector in binary format:')
array_to_string(vector['inf'])

For information vector in binary format:


'101101111010010011010001101111011000101010001010100101010110010111011001100111001100011000001000110011010001110110010000011110111101000101010101010010111110000111000001011111010001110101011001011010111111011000010001101001010111100001100111010111100111100100100111000100010110110000000001000111110011100011010101011010011010101001000100111100111110011011110101110111000001011010001101101001111011011100001101100110001101011010111000111100001010000010101010111100011010001101010000000110011000011000101100111100110110110001100001010001001110100001101001110001011001000000100001011010111100101011111111000110001010101101011010111011000000001111011000100000000111100101011111100001011010000111101111101101001010001111111001100001101110000010111001100101000000011000000001101011110000001100110010110011010110001110100000110100100010011011010000111101010110001111101000101101110001100110111000000100110000000001100101000100111000011111'

In [4]:
print('For information vector in hex format:')
bin_to_hex(vector['inf'])


For information vector in hex format:


'de52b8db15159a6ab99336013b8b90edb8aa2d7838eb8ba96df6885ae16ea7e94e8863088fc1ba695522fc76fab3861b5ede0b91b6d1f05055f85ca0891643fc63682271693a90486d35ff815da5730cb110e9af1a587fd25cf91670d99206085f0cc43b6c50b446b0fa6c71de89d18c006a8c1e3'

In [5]:
print('For error vector:')
array_to_string(vector['err'])

For error vector:


'000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000